In [1]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()
import re

In [2]:
stone = pd.read_excel("../data/musicbrainzset.xlsx")

In [3]:
#como hay nombres de álbumes que son números, los convertimos a string.
def num_string(dato):
    if type(dato)== int:
        return str(dato)
    else:
        return dato

In [4]:
stone["Album"] = stone["Album"].apply(num_string)

In [5]:
stone.Artist = stone.Artist.str.strip()
stone.Album = stone.Album.str.strip()
stone.Type = stone.Type.str.strip()
stone.Genre = stone.Genre.str.strip()

In [6]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]
stone["Subgenre"] = stone["Genre"].str.extract(r",\s*(.*$)")[0]

In [7]:
stone.Gen = stone.Gen.str.strip()
stone.Subgenre = stone.Subgenre.str.strip()

In [8]:
stone.Rating.fillna(0,inplace = True)
stone.Subgenre.fillna("None",inplace = True)

In [9]:
#hay algunos títulos de album que hay que cambiar de nombre
stone.Album = stone.Album.str.replace(r'(Metallica...The Black Album..)',"Metallica",regex=True)
stone.Album = stone.Album.str.replace(r'(Eagles...st album.)',"Eagles",regex=True)
stone.Artist = stone.Artist.str.replace("Neil Young with Crazy Horse","Neil Young & Crazy Horse",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Bob Dylan...The Band)","Bob Dylan And The Band",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Prince\s+.*)","Prince & The Revolution",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Rufus...Chaka Khan)","Rufus",regex=True)
stone.Album = stone.Album.str.replace(r'(Proud Mary.*)',"Best Of / Proud Mary",regex=True)
stone.Artist = stone.Artist.str.replace(r'(.*The Velvet Underground.*)',"The Velvet Underground",regex= True)
stone.Artist = stone.Artist.str.replace("‐","-",regex = True)
stone.Album = stone.Album.str.replace("‐","-",regex = True)
stone.Gen = stone.Gen.str.replace("piunk","punk",regex = True)
stone.Gen = stone.Gen.str.replace("punk rock","punk",regex = True)
stone.Gen = stone.Gen.str.replace("folk rock","folk",regex = True)

In [10]:
#creamos nuevas columnas que haremos compatibles con url
stone["Alb_url"] = stone["Album"]
stone["Art_url"] = stone["Artist"]

In [11]:
#algunos nombres de discos hay que cambiarlos
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Band...The Brown Album..)',"The Band",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^Sign.*the Times$)',"Sign 'O' The Times",regex=True)

In [12]:
#función que cambia los caracteres especiales para las urls
def url_prep(df, colum):
    url_replace = {"'":"%27",",":"%2c", "&":"%26",".":"%2e","/":"%2f","#":"%23","(":"%28",")":"%29","-":"%2d",'"':"%22",r"(\s+)":"%20" }
    for key,value in url_replace.items():
        df[colum] = df[colum].str.replace(key,value,regex=True)

In [13]:
df = stone
colum = "Alb_url"
url_prep(df,colum)
colum = "Art_url"
url_prep(df,colum)

In [14]:
stone.sample(3)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
5,6,1991,Nevermind,Nirvana,Album,4.45,68,grunge,grunge,None,Nevermind,Nirvana
73,74,2003,The College Dropout,Kanye West,Album,4.40,14,hip-hop,hip-hop,None,The%20College%20Dropout,Kanye%20West
393,394,1980,Diana,Diana Ross,Album,5.00,14,"soul , disco",soul,disco,Diana,Diana%20Ross


In [15]:
apikey = os.getenv("apikey") #apikey de lastfm

In [16]:
#comprobamos respuestas de las urls
def urls_llamadas (df,art,alb):
    api_urls = []
    apikey = os.getenv("apikey")
    for s in range(len(df)):
        artist = stone.loc[s,art]
        album = stone.loc[s,alb]
        api_urls.append(f"http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={apikey}&artist={artist}&album={album}&format=json")
    request_dic = [] 
    i = 0
    for a in api_urls:
        res = requests.get(a).json()
        request_dic.append(res["album"])
        if i%50 == 0:
            print(f"{i} done")
        elif i == (len(api_urls)-1):
            print(f"{i} done,finished")
        i+=1
    return request_dic

In [17]:
df = stone
art = "Art_url"
alb = "Alb_url"
Top500_last = urls_llamadas (df,art,alb)

0 done
50 done
100 done
150 done
200 done
250 done
300 done
350 done
400 done
450 done
499 done,finished


In [18]:
stone.to_csv("../data/stone.csv",index=False)

In [19]:
Top500_last_df = pd.DataFrame(Top500_last)

In [20]:
Top500_last_df.sample(3)

,artist,mbid,tags,playcount,image,tracks,url,name,listeners,wiki
453,Can,022ccb24-be89-43d3-8fc9-e9421ae3ac9b,{'tag': [{'url': 'https://www.last.fm/tag/krau...,2093050,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/Can/Ege+Bamyasi,Ege Bamyasi,146371,"{'published': '01 Jan 2013, 02:09', 'summary':..."
202,Nick Drake,278903c5-465e-4dfe-b278-10e7ce02f0d3,{'tag': [{'url': 'https://www.last.fm/tag/folk...,20845854,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/Nick+Drake/Pink+Moon,Pink Moon,961471,"{'published': '01 Jan 2012, 12:02', 'summary':..."
472,Daddy Yankee,0b0e84e9-a01f-41bc-bc5b-4d0322983076,{'tag': [{'url': 'https://www.last.fm/tag/regg...,1200306,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/Daddy+Yankee/Barrio+...,Barrio Fino,199529,NaN


In [21]:
Top500_last_df.to_csv("../data/otras/Top500LastFM_tracks.csv",index=False) #meloguardoporsiacaso

In [22]:
Top500_last_df.sample()

,artist,mbid,tags,playcount,image,tracks,url,name,listeners,wiki
319,X,24652d4d-c0fc-452f-9f05-c8992bcfaf08,{'tag': [{'url': 'https://www.last.fm/tag/punk...,1436983,"[{'size': 'small', '#text': 'https://lastfm.fr...","{'track': [{'streamable': {'fulltrack': '0', '...",https://www.last.fm/music/X/Los+Angeles,Los Angeles,141553,"{'published': '10 Oct 2008, 15:40', 'summary':..."


In [23]:
Top500_last_df[["playcount","listeners"]] = Top500_last_df[["playcount","listeners"]].astype("int64")

In [24]:
Lastfm_500 = Top500_last_df[["artist","playcount","name","listeners"]]
Lastfm_500.head(3)

,artist,playcount,name,listeners
0,Marvin Gaye,5225169,What's Going On,544344
1,The Beach Boys,13405485,Pet Sounds,543490
2,Joni Mitchell,9352559,Blue,535371


In [25]:
stone_500 = stone[["Number","Year","Album","Artist","Type","Rating","Gen"]]
stone_500.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen
393,394,1980,Diana,Diana Ross,Album,5.00,soul
382,383,1998,Mezzanine,Massive Attack,Album,4.35,acid house
470,471,1967,Surrealistic Pillow,Jefferson Airplane,Album,4.30,rock


In [26]:
stone.Gen.value_counts()

rock           164
hip-hop         58
pop             56
folk            33
r&b             28
soul            27
country         19
punk            18
alternative     14
funk            13
jazz            11
new wave         9
blues            9
metal            6
reggae           5
grunge           4
disco            4
electronic       4
latin            3
fusion           2
dance            2
acid house       2
ambient          2
indie            2
gospel           1
post punk        1
emo              1
juju             1
mbaqanga         1
Name: Gen, dtype: int64

In [27]:
stone_500_richment = stone_500.merge(Lastfm_500,left_index=True, right_index=True)
stone_500_richment.head(3)

,Number,Year,Album,Artist,Type,Rating,Gen,artist,playcount,name,listeners
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,r&b,Marvin Gaye,5225169,What's Going On,544344
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,rock,The Beach Boys,13405485,Pet Sounds,543490
2,3,1971,Blue,Joni Mitchell,Album,4.45,folk,Joni Mitchell,9352559,Blue,535371


In [28]:
stone_500_rich =stone_500_richment [["Number","Year","Album","Artist","Type","Rating","Gen","playcount","listeners"]]
stone_500_rich.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
315,316,1967,The Who Sell Out,The Who,Album,4.00,rock,2526236,142862
72,73,1991,Loveless,My Bloody Valentine,Album,4.55,alternative,31928130,886008
211,212,1966,Wild Is the Wind,Nina Simone,Album,5.00,jazz,967027,103072


In [32]:
stone_500_rich.sort_values("listeners").head(10)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
464,465,2003,The Best of the Classic Years,King Sunny Adé,Album + Compilation,0.0,juju,1571,115
488,489,1991,Back to Mono (1958–1969),Phil Spector,Album + Compilation,0.0,pop,5558,183
404,405,1972,Nuggets: Original Artyfacts From the First Psy...,Various Artists,Album + Compilation,0.0,rock,7332,244
162,163,1977,Saturday Night Fever: The Original Movie Sound...,Various Artists,Album + Soundtrack,5.0,disco,6999,434
355,356,1968,Gris-Gris,"Dr. John, the Night Tripper",Album,3.0,rock,5409,491
463,464,1973,3+3,The Isley Brothers,Album,4.0,funk,1061,495
324,325,1993,"All Killer, No Filler: The Anthology",Jerry Lee Lewis,Album + Compilation,0.0,rock,10748,525
401,402,1975,Expensive Shit,Fela Ransome Kuti & Africa 70,Album,4.0,funk,2827,550
346,347,1995,Liquid Swords,Genius/GZA,Album,4.0,hip-hop,9750,905
496,497,1985,The Indestructible Beat of Soweto,Various Artists,Album + Compilation,0.0,mbaqanga,10677,911


In [33]:
stone_500_rich.to_csv("../data/stone_500_rich.csv",index=False)

In [34]:
stone_500_rich.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Number     500 non-null    int64  
 1   Year       500 non-null    int64  
 2   Album      500 non-null    object 
 3   Artist     500 non-null    object 
 4   Type       500 non-null    object 
 5   Rating     500 non-null    float64
 6   Gen        500 non-null    object 
 7   playcount  500 non-null    int64  
 8   listeners  500 non-null    int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 35.3+ KB
